In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import torch 
torch.cuda.empty_cache()
torch.cuda.device_count()

3

In [3]:
import os
import torch
from datasets import load_dataset, load_from_disk
from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
import nltk
from peft import LoraConfig, get_peft_model, prepare_model_for_int8_training, TaskType
import evaluate
from typing import List, Dict

2024-04-03 21:10:18.707037: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-03 21:11:07.464904: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/slurm/lib64:/opt/slurm/lib64:
2024-04-03 21:11:07.464942: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-04-03 21:11:14.099108: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-03 21:1

In [4]:
train_df = load_from_disk("/home/y.khan/cai6307-y.khan/Query-Focused-Tabular-Summarization/data/decomposed/decomposed_train")
test_df = load_from_disk("/home/y.khan/cai6307-y.khan/Query-Focused-Tabular-Summarization/data/decomposed/decomposed_test")
validate_df = load_from_disk("/home/y.khan/cai6307-y.khan/Query-Focused-Tabular-Summarization/data/decomposed/decomposed_validate")

In [5]:
test_example_ids = set(test_df['example_id'])
validate_example_ids = set(validate_df['example_id'])
common_example_ids = test_example_ids.intersection(validate_example_ids)

test_df = test_df.filter(lambda example: example['example_id'] not in common_example_ids)
test_df

Filter: 100%|██████████| 500/500 [00:00<00:00, 1714.39 examples/s]


Dataset({
    features: ['row_ids', 'table', 'summary', 'query', 'example_id', 'coordinates', 'answers'],
    num_rows: 300
})

In [6]:
# model="google/flan-t5-xxl"
model= "Flan-xxl-lora"

cache_dir = "/home/y.khan/cai6307-y.khan/Query-Focused-Tabular-Summarization/models/llama"

# Load tokenizer of FLAN-t5-XL
tokenizer = AutoTokenizer.from_pretrained(model, cache_dir=cache_dir)
# Load the model
model = AutoModelForSeq2SeqLM.from_pretrained(model, cache_dir=cache_dir, load_in_8bit=True, device_map="auto")

config.json: 100%|██████████| 674/674 [00:00<00:00, 3.66MB/s]
model.safetensors.index.json: 100%|██████████| 53.0k/53.0k [00:00<00:00, 2.56MB/s]
model-00001-of-00005.safetensors:   0%|          | 0.00/9.45G [00:00<?, ?B/s]
model-00001-of-00005.safetensors:   0%|          | 10.5M/9.45G [00:00<03:27, 45.6MB/s]
model-00001-of-00005.safetensors:   0%|          | 31.5M/9.45G [00:00<03:26, 45.6MB/s]
model-00001-of-00005.safetensors:   1%|          | 52.4M/9.45G [00:01<02:56, 53.3MB/s]
model-00001-of-00005.safetensors:   1%|          | 62.9M/9.45G [00:01<02:34, 60.7MB/s]
model-00001-of-00005.safetensors:   1%|          | 83.9M/9.45G [00:01<02:40, 58.5MB/s]
model-00001-of-00005.safetensors:   1%|          | 94.4M/9.45G [00:01<02:45, 56.5MB/s]
model-00001-of-00005.safetensors:   1%|          | 105M/9.45G [00:01<02:44, 56.7MB/s] 
model-00001-of-00005.safetensors:   1%|▏         | 126M/9.45G [00:02<02:02, 76.2MB/s]
model-00001-of-00005.safetensors:   1%|▏         | 136M/9.45G [00:02<03:31, 44.0MB

In [7]:
def tokenization_with_answer(examples):
    inputs = []
    
    task_prefix = "Given a query and a table, generate a summary that answers the query based on the information in the table: "

    for i, (query, table, summary) in enumerate(zip(examples['query'], examples['table'], examples['summary'])):
        flattened_table = flatten_table(table, i)
        input_text = f"{task_prefix} Table {flattened_table}. Query: {query}"

        inputs.append(input_text)
        
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True,padding='max_length')
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"], max_length=512, truncation=True)
    
    model_inputs["labels"] = labels["input_ids"] 

    return model_inputs


def flatten_table(table: Dict, row_index: int) -> str:
    header = table.get('header', [])
    rows = table.get('rows', [])
    title = table.get('title', [])

    flattened_rows = []
    for i, row in enumerate(rows):
        row_text = f"Row {i}, " + ",".join([f"{col}:{val}" for col, val in zip(header, row)])
        flattened_rows.append("## "+row_text)

    flattened_table = f"Title: {' '.join(map(str, title))}" + " " + " ".join(flattened_rows)
    return flattened_table

tokenized_dataset_train = train_df.map(tokenization_with_answer, batched=True)
tokenized_dataset_test = test_df.map(tokenization_with_answer, batched=True)

processed_data_train = tokenized_dataset_train.remove_columns(['table','summary', 'row_ids', 'example_id', 'query', 'answers', 'coordinates'])
processed_data_test = tokenized_dataset_test.remove_columns(['table','summary', 'row_ids', 'example_id', 'query', 'answers', 'coordinates'])

Map: 100%|██████████| 300/300 [00:02<00:00, 110.49 examples/s]


In [8]:
processed_data_train

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 2000
})

In [9]:
processed_data_train = processed_data_train.shuffle().select(range(1024))
processed_data_test = processed_data_test.shuffle().select(range(200))

In [10]:
# Define LoRA Config
lora_config = LoraConfig(
 r=16,
 lora_alpha=64,
 target_modules=["q_proj", "up_proj", "o_proj", "k_proj", "down_proj", "gate_proj","v_proj", "ln", "fc", "q", "v", "lm_head"],
 lora_dropout=0.05,
 bias="none",
 task_type=TaskType.SEQ_2_SEQ_LM
)
# prepare int-8 model for training
model = prepare_model_for_int8_training(model)

# add LoRA adaptor
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 19,453,952 || all params: 11,154,786,304 || trainable%: 0.1744000420072951


In [11]:
from transformers import DataCollatorForSeq2Seq

# we want to ignore tokenizer pad token in the loss
label_pad_token_id = -100
# Data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)

In [12]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

def postprocess_text(preds, labels):
        preds = [pred.strip() for pred in preds]
        labels = [label.strip() for label in labels]

        # rougeLSum expects newline after each sentence
        preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
        labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]

        return preds, labels

def metric_fn(eval_predictions):
    predictions, labels = eval_predictions
    decoded_predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    for label in labels:
        label[label < 0] = tokenizer.pad_token_id  # Replace masked label tokens
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    decoded_predictions, decoded_labels = postprocess_text(decoded_predictions, decoded_labels)

    rouge = evaluate.load('rouge')

    # Compute ROUGE scores
    rouge_results = rouge.compute(predictions=decoded_predictions, references=decoded_labels)

    return rouge_results

output_dir="lora-flan-t5-xxl"

# Define training args
training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,
    save_strategy = "no",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=8,
    learning_rate=2e-4, 
    num_train_epochs = 20,
    gradient_accumulation_steps=2,
    optim="paged_adamw_32bit",
    save_steps=-1,
    logging_steps=25,
    weight_decay=0.001,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    load_best_model_at_end=True,
    predict_with_generate=True,
    overwrite_output_dir= True,
    lr_scheduler_type="constant",
    report_to="tensorboard",
)

# Create Trainer instance
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=processed_data_train,
    eval_dataset=processed_data_test,
    compute_metrics=metric_fn,
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!


In [13]:
trainer.train()

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
25,1.590100
50,1.414200
75,1.345400
100,1.293900
125,1.233700
150,1.207800
175,1.160500
200,1.125000
225,1.073700
250,1.042600


TrainOutput(global_step=640, training_loss=0.9859529733657837, metrics={'train_runtime': 36411.5682, 'train_samples_per_second': 0.562, 'train_steps_per_second': 0.018, 'total_flos': 1.3870377839099904e+18, 'train_loss': 0.9859529733657837, 'epoch': 20.0})

In [14]:
model.save_pretrained("Flan-xxl-lora")
tokenizer.save_pretrained("Flan-xxl-lora")

('Flan-xxl-lora/tokenizer_config.json',
 'Flan-xxl-lora/special_tokens_map.json',
 'Flan-xxl-lora/spiece.model',
 'Flan-xxl-lora/added_tokens.json',
 'Flan-xxl-lora/tokenizer.json')

In [13]:
validate_df = load_from_disk("/home/y.khan/cai6307-y.khan/Query-Focused-Tabular-Summarization/data/decomposed/decomposed_validate")
validate_df_size = len(validate_df)
step_size = 3
num_batches = validate_df_size // step_size

valid = []

for i in range(num_batches):
    start_index = i * step_size
    end_index = (i + 1) * step_size
    valid.append(validate_df.select(range(start_index, end_index)))

# If there are remaining data points that don't fit into full batches of size 3
if validate_df_size % step_size != 0:
    remaining_data = validate_df_size % step_size
    valid.append(validate_df.select(range(validate_df_size - remaining_data, validate_df_size)))

In [14]:
rougeL = []
for i in range(len(valid)):
    validate_df = valid[i].map(tokenization_with_answer, batched=True)
    predict_results = trainer.predict(validate_df, max_length = 512)
    metrics = predict_results.metrics

    rougeL.append(metrics['test_rougeLsum'])

sum(rougeL)/len(rougeL)

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Map: 100%|██████████| 3/3 [00:00<00:00, 180.81 examples/s]


Map: 100%|██████████| 3/3 [00:00<00:00, 195.34 examples/s]


Map: 100%|██████████| 3/3 [00:00<00:00, 232.93 examples/s]


Map: 100%|██████████| 3/3 [00:00<00:00, 199.72 examples/s]


...

Map: 100%|██████████| 3/3 [00:00<00:00, 207.63 examples/s]


Map: 100%|██████████| 3/3 [00:00<00:00, 198.27 examples/s]


Map: 100%|██████████| 3/3 [00:00<00:00, 257.39 examples/s]


Map: 100%|██████████| 2/2 [00:00<00:00, 150.89 examples/s]


0.29763254934970884

In [15]:
rougeL

[0.3248111797730118,
 0.5381331978797139,
 0.22295943050660028,
 0.28098919054801413,
 0.29505790620341704,
 0.236795157693533,
 0.29520026481297584,
 0.2535331845676673,
 0.26100576955950355,
 0.27375983652884656,
 0.26382657746294114,
 0.31554027113237637,
 0.30985045111852677,
 0.3161806030227083,
 0.4032217340806949,
 0.45417802924110745,
 0.29725072787783685,
 0.1010035892524856,
 0.17429078323590144,
 0.13852118050817044,
 0.2764951888104597,
 0.2557482255939479,
 0.21302232350728248,
 0.256602086438152,
 0.39433551198257083,
 0.17430393412166403,
 0.7085160942033876,
 0.49346538235427123,
 0.2717236985802694,
 0.27684526343530175,
 0.262487803717189,
 0.15206972283657824,
 0.2428571428571428,
 0.2382944584437122,
 0.2800055490272882,
 0.34647426647426643,
 0.3806438609805334,
 0.17436433298502263,
 0.38004949552636585,
 0.38926200856211457,
 0.4047055561723272,
 0.2234174059731869,
 0.427884688754254,
 0.31158280922431864,
 0.23947308653191005,
 0.22440838404327246,
 0.283677399

In [17]:
validate_df = valid[20].map(tokenization_with_answer, batched=True)
predict_results = trainer.predict(validate_df, max_length = 1024)
metrics = predict_results.metrics
metrics['test_rougeLsum']

0.2764951888104597

In [18]:
predictions = tokenizer.batch_decode(predict_results.predictions, skip_special_tokens=True, clean_up_tokenization_spaces=True)
predictions = [pred.strip() for pred in predictions]
predictions

['The three oldest Southern League stadiums are: Regions Field, AT&T Field, and The Ballpark at Jackson.',
 'The 1892 AHAC season had a total of 8 Exhibition matches, with Montreal Hockey Club playing in 5 of them.',
 'The best batting average (minimum 80 at bats - ab) from the Forest City Owls was.286 in 2010 and the best season result was Won League Championship.']

In [19]:
validate_df['summary']

['The three oldest Southern League stadium are Hank Aaron Stadium, The Ballpark at Jackson, and AT&T Field. The Mobile BayBears connect with Hank Aaron Stadium, open in 1997 in Mobile, Alabama, when the Jackson Generals play at The Ballpark at Jackson, this stadium in Jackson, Tennessee was establish in 1998. Lastly, the Chattanooga Lookouts link to AT&T Field, it in Chattanooga, Tennessee, and open in 2000. These stadiums giving a historic home for their baseball teams in the Southern League.',
 "A total of 6 'Exhibition' type matches be hold in 1892 AHAC season from record data. Montreal Hockey Club play most in this type matches, with 4 exhibition games.",
 'The Forest City Owls, they have some good perform in batting average in years. The best perform is in 2015 by Spencer Henderson, who get great batting average of .398; but team no go playoffs that year. Second-best perform in batting average is in 2009 by Konstantine Diamaduros, with average of .376, which helps team to win Leag